In [1]:
from glob import glob

In [2]:
import json
import requests
import os
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
pathOut = "~/data/ocean/source/"
pathOut = os.path.expanduser(pathOut)

if not os.path.exists(pathOut):
    os.makedirs(pathOut)

In [7]:
os.path.exists('/home/davemlz/data/ocean/source/monthly_global_marine_phytoplankton_primary_production/BICEP_NCEO_PP_ESA-OC-L3S-MERGED-1M_MONTHLY_9km_mapped_199801-fv4.2.nc')

True

In [4]:
glob('/home/davemlz/data/ocean/source/monthly_global_particulate_organic_carbon_(poc)/')

['/home/davemlz/data/ocean/source/']

In [69]:
with open('../cube9km.geojson', 'r') as f:
    cube_specs = json.load(f)

In [70]:
# cube_specs

In [71]:
def list_files_nc(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

In [72]:
years = np.arange(1997,2021)
url="https://data.ceda.ac.uk/neodc/bicep/data/marine_primary_production/v4.2/monthly/"
files = []
for year in tqdm(years):
    r = requests.get(f"{url}{year}")
    if r.status_code == 200:
        files.extend(list_files_nc(f"{url}{year}/","download=1"))

100%|██████████| 24/24 [00:11<00:00,  2.06it/s]


276

In [ ]:
def download_file(source):
    output_source_path = pathOut + "/" + source['name'].lower().replace(" ","_")
    if not os.path.exists(output_source_path):
        os.makedirs(output_source_path)
    url = source['download_url']
    years = np.arange(1997,2021)
    files = []
    for year in tqdm(years):
        r = requests.get(f"{url}{year}")
        if r.status_code == 200:
            files.extend(list_files_nc(f"{url}{year}/","download=1"))
    files = np.unique(files).tolist()
    for file in files:
        response = requests.get(file)
        filename = file.replace("?download=1","")
        output = output_source_path + "/" + filename
        open(output, "wb").write(response.content)

In [ ]:
                
[download_file(source) for source in tqdm(cube_specs['properties']['sources'])]